# Strict Linking - ID

> This is very simple linking based on exact ID match between all of the datasets.

In [1]:
import json
import pandas
import time
import os
import csv
from functools import reduce
import functions as fx
import core_constants as cc
import pandas as pd
import sqlite3 as sql

### Load the Summarized Files

> These are files that only return ID + the fields that I want in the final printout

In [ ]:
summarizedDir = '..//summarizedData//'

## Load the source file dict
summarizedFiles = json.loads(open('..//config//summarizedFiles.json', "r").read())

### Process the Summarized Files

> We are going to put these files into a DataFrame and then add them all to a single list.  The print out then is all of the keys we are processing per the summarizedFiles.json file.

1. Load IDs of target dataset, along with keydataset into separate dataframes
2. Use the code below to join the dataframes - there will be column name collisions
3. Take the dictionary - append the manual KeyLinkType, KeyDataSet, possibly a set Confidence level?
4. Insert the dictionary into the RecordLinks database table
5. Remove the CSV write
6. Figure out how Views work and make sure to trigger a rewrite of the view with all of the expected views of the final dataset

dfs = []
keys = ['Sports247', 'AllConference', 'AllAmerican', 'NFL', 'NCAA']
for item in keys:
    SQL_Query = 'SELECT ID, school FROM ' + item
    vars()[item] = fx.connDBAndReturnDF(SQL_Query)
    dfs.append(vars()[item])

In [ ]:
Sports247.head()

,ID
0,paulthurston_nebraska
1,gregmcmullen_nebraska
2,jordanwesterkamp_nebraska
3,imanicross_nebraska
4,tommyarmstrong_nebraska


dfs = []
for key in summarizedFiles.keys():
    vars()[key] = pandas.DataFrame(json.loads(open(summarizedDir + summarizedFiles[key][0], "r", encoding="utf-8").read()))
    dfs.append(vars()[key])
    

### Run the Strict Linking on ID

> This is so cool - first time I've ever used lambda as well.  Compares each DFS by a similar field then joins them all into a single DF.  

###### Src: https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns

In [ ]:
df_final = reduce(lambda left,right: pandas.merge(left,right,how="left", on='ID'), dfs)
df_final.head()

### Save it to a CSV

> Ideally this will all write to a db eventually but for now we are writing out to a CSV file for import to a stats program

In [ ]:
with open("..//linkedPlayers.csv", "w", encoding="utf-8") as write_file:
                write_file.write(df_final.to_csv())

# SCRATCHPAD!

> The below needs to be cleaned up but it works and successfully writes out all fo the records to the RecordLinks table.

In [5]:

conn = sql.connect(cc.databaseName)
c = conn.cursor()
print("Connected to SQLite")


keys = ['NCAA']
for key in keys:
    ## Get the KeyDataSet
    d_tuple = [key]
    dataset = c.execute('select KeyDataSet from DataSet where DataSet = ?', d_tuple)
    keydataset = c.fetchone()
    ## Load all of the IDs
    dataset_tuple = [keydataset[0]]
    print(keydataset)
    if(keydataset[0] != 5):
        fetchIds = c.execute('SELECT a.ID from SourcedPlayers a inner join SourcedPlayers b on (a.ID = b.ID and b.KeyDataSet = 1) where a.KeyDataSet = ?', dataset_tuple)
    else:
        fetchIds = c.execute('SELECT DISTINCT a.ID from SummarizedNCAAData a inner join SourcedPlayers b on (a.ID = b.ID and b.KeyDataSet = 1)')
    records = c.fetchall()
    print(len(records))
    
    ## Insert records into the RecordLinks table, 
    for record in records:
        sqlite_insert_query = """INSERT INTO RecordLinks
                              (MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence) 
                               VALUES 
                              (?,?,?,2,1);"""
        data_tuple = [record[0],record[0],keydataset[0]]
        count = c.execute(sqlite_insert_query, data_tuple)
        conn.commit()


Connected to SQLite
(5,)
24949
